Setup

In [26]:

import os, types
from dotenv import load_dotenv
load_dotenv()

f = open("test.txt", "r")



In [27]:
document = f.read()
document

'How do I know which activity codes to use when claiming in either ILC or CATS? \ue278\nRefer to the Activity Code model found on the Claiming Basics tab of this training module.\nLabor should be accurately reflected according to the type of work performed including billable, customer facing, enablement, time away,\netc.\nWhat if I need to enter hours in both ILC and CATS? \ue278\nHours should be worked within the claim system according to the country which owns the contract. In order to claim labor within ILC, an EMF record is required. In order to claim labor in CATS, a Mini-master record is required. Practitioners can certainly split their time between systems, as needed for contract labor, for a given week. Please refer to the Labor Claiming reference page for information.\nWhat if I am located in a “Blue Harmony” country that is required to record time in CATS (the SAP time recording application)?\n\ue278\nWho controls which activity codes are available on the work items I use for

In [28]:
import getpass
import os

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : os.getenv('API_KEY')
	}

In [29]:
#model_id = "meta-llama/llama-2-13b-chat"
model_id = "ibm-mistralai/mixtral-8x7b-instruct-v01-q"
#model_id = "google/flan-ul2"

In [30]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 2000,
    "stop_sequences": ["<eof>"],
    "repetition_penalty": 1
}

In [31]:
project_id = os.getenv("watsonx_project_id")
space_id = None
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    pass
try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    pass


In [32]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id = model_id,
    params = parameters,
    credentials = get_credentials(),
    project_id = project_id,
    space_id = space_id
)

In [33]:
def gen_faq_content_prompt(content):
    return f"""<s> [INST] <<SYS>>
    as a data curator, you must generate a pair of questions and answers based on a document /
    All of the content what you will generate the pair of questions and answers its on the document /
    You do not generate any informations out of the document

    Your output must be a valid JSON file. follow the rules below

    {{
        question id: must have a number for any pair of questions and answers
        question: Generate a question and your answers from the document and only the document
        answer: Retrieve the answer of the question from the document
    }}
<</SYS>>

Document: {content}   
 
    Output:
    [/INST]
    """

In [34]:
prompt_input = gen_faq_content_prompt(document)

In [35]:
print(prompt_input)

<s> [INST] <<SYS>>
    as a data curator, you must generate a pair of questions and answers based on a document /
    All of the content what you will generate the pair of questions and answers its on the document /
    You do not generate any informations out of the document

    Your output must be a valid JSON file. follow the rules below

    {
        question id: must have a number for any pair of questions and answers
        question: Generate a question and your answers from the document and only the document
        answer: Retrieve the answer of the question from the document
    }
<</SYS>>

Document: How do I know which activity codes to use when claiming in either ILC or CATS? 
Refer to the Activity Code model found on the Claiming Basics tab of this training module.
Labor should be accurately reflected according to the type of work performed including billable, customer facing, enablement, time away,
etc.
What if I need to enter hours in both ILC and CATS? 
Hours should

In [36]:
generated_response = model.generate_text(prompt=prompt_input)
print(generated_response)


    [
        {
            "question_id": 1,
            "question": "How should labor be accurately reflected in the claim systems?",
            "answer": "Labor should be accurately reflected according to the type of work performed including billable, customer facing, enablement, time away, etc."
        },
        {
            "question_id": 2,
            "question": "What is required to claim labor within ILC and CATS?",
            "answer": "In order to claim labor within ILC, an EMF record is required. In order to claim labor in CATS, a Mini-master record is required."
        },
        {
            "question_id": 3,
            "question": "Who manages the work item request process for labor claiming?",
            "answer": "There are CLAIM administration teams in each Geo / Market / Country who manage the work item request process."
        },
        {
            "question_id": 4,
            "question": "Under what circumstances can hours related to a training class